In [1]:
#pip install --upgrade ipywidgets

In [2]:
#pip install --upgrade tqdm

In [1]:
import torch  # Importowanie biblioteki PyTorch
import torch.nn as nn  # Importowanie modułu do tworzenia sieci neuronowych
import torch.optim as optim  # Importowanie modułu do optymalizacji
import torchvision  # Importowanie biblioteki torchvision
import torchvision.transforms as transforms  # Importowanie modułu do transformacji danych

# Przykład wczytywania i przetwarzania danych
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Zmniejszenie rozmiaru obrazów do 64x64 pikseli
    transforms.ToTensor(),  # Konwersja obrazów na tensory
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizacja tensorów
])

# Wczytujemy dane
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)  # Wczytanie zbioru danych CIFAR-10 i zastosowanie transformacji
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)  # Tworzenie dataloadera do trenowania

# Definiujemy model
class CNN(nn.Module):  # Definicja klasy modelu CNN
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Pierwsza warstwa konwolucyjna
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # Druga warstwa konwolucyjna
        self.fc1 = nn.Linear(64 * 16 * 16, 128)  # Warstwa fully connected (w pełni połączona)
        self.fc2 = nn.Linear(128, 10)  # Warstwa fully connected (w pełni połączona)

    def forward(self, x):
        x = torch.relu(self.conv1(x))  # Przepływ przez pierwszą warstwę konwolucyjną z funkcją aktywacji ReLU
        x = torch.max_pool2d(x, 2)  # Max pooling
        x = torch.relu(self.conv2(x))  # Przepływ przez drugą warstwę konwolucyjną z funkcją aktywacji ReLU
        x = torch.max_pool2d(x, 2)  # Max pooling
        x = x.view(-1, 64 * 16 * 16)  # Spłaszczenie tensora
        x = torch.relu(self.fc1(x))  # Przepływ przez pierwszą warstwę fully connected z funkcją aktywacji ReLU
        x = self.fc2(x)  # Przepływ przez drugą warstwę fully connected
        return x

# Inicjalizacja modelu
model = CNN()

# Trenowanie modelu
criterion = nn.CrossEntropyLoss()  # Wybór funkcji straty CrossEntropy
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Wybór optymalizatora Adam z wybranym learning rate

for epoch in range(2):  # Przykładowa liczba epok
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()  # Resetowanie gradientów
        outputs = model(inputs)  # Przepływ danych przez model
        loss = criterion(outputs, labels)  # Obliczenie funkcji straty
        loss.backward()  # Wsteczna propagacja
        optimizer.step()  # Aktualizacja wag modelu
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')  # Wyświetlenie bieżącej straty

print('Trening zakończony.')

# Zapis modelu
torch.save(model.state_dict(), 'model.pth')  # Zapisanie stanu modelu do pliku 'model.pth'


Files already downloaded and verified
Epoch 1, Loss: 1.3038601733062485
Epoch 2, Loss: 0.9407494016663813
Trening zakończony.
